# VacationPy
----

#### My Observations
* ....

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

gmaps.configure(api_key = g_key)

In [2]:
# gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
data = pd.read_csv('../OutputData/cities.csv')
data

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.19,168.86,49.84,78,30,7.07,NZ,1594435433
1,Saldanha,-33.01,17.94,51.80,81,75,12.75,ZA,1594435451
2,Arraial do Cabo,-22.97,-42.03,69.55,76,32,16.62,BR,1594435425
3,Bredasdorp,-34.53,20.04,50.00,81,70,16.11,ZA,1594435434
4,Tuktoyaktuk,69.45,-133.04,42.80,87,75,12.75,CA,1594435452
...,...,...,...,...,...,...,...,...,...
537,Ocós,14.51,-92.19,75.20,100,40,2.24,GT,1594435613
538,Barentu,15.11,37.59,73.67,74,3,5.82,ER,1594435614
539,Owando,-0.48,15.90,71.55,95,100,2.66,CG,1594435614
540,Chara,56.91,118.26,85.55,29,1,17.49,RU,1594435614


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
lat_long = data[['Latitude', 'Longitude']]
lat_long

,Latitude,Longitude
0,-46.19,168.86
1,-33.01,17.94
2,-22.97,-42.03
3,-34.53,20.04
4,69.45,-133.04
...,...,...
537,14.51,-92.19
538,15.11,37.59
539,-0.48,15.90
540,56.91,118.26


In [5]:
hum = data['Humidity']
hum

0       78
1       81
2       76
3       81
4       87
      ... 
537    100
538     74
539     95
540     29
541     72
Name: Humidity, Length: 542, dtype: int64

In [6]:
# type(humidity[0])

In [7]:
fig = gmaps.figure(zoom_level=2, center=(20,30))
heat_layer = gmaps.heatmap_layer(lat_long, weights=hum, dissipating=False, max_intensity=10, point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_weather = data.loc[(data['Max Temperature'] > 70) & 
                                 (data['Max Temperature'] < 80) & 
                                 (data['Wind Speed'] < 10) & 
                                 (data['Cloudiness'] == 0)]
ideal_weather

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
106,Awjilah,29.11,21.29,72.18,62,0,8.61,LY,1594435388
113,Mednogorsk,51.41,57.59,79.95,56,0,1.97,RU,1594435515
141,Marsá Maţrūḩ,31.35,27.25,71.60,83,0,5.82,EG,1594435522
214,Jalu,29.03,21.55,72.28,62,0,8.10,LY,1594435541
225,Komsomolets,53.75,62.07,79.97,65,0,4.27,KZ,1594435543
302,Corinth,37.94,22.96,74.52,44,0,1.01,GR,1594435561
360,Al Bardīyah,31.76,25.09,71.01,81,0,8.59,LY,1594435463
371,Lazurne,46.08,32.53,72.57,66,0,7.45,UA,1594435576
378,Derzhavīnsk,51.10,66.32,76.37,57,0,7.54,KZ,1594435578
444,Ben Gardane,33.14,11.22,72.41,82,0,5.01,TN,1594435592


In [9]:
ideal_weather.count()

City               10
Latitude           10
Longitude          10
Max Temperature    10
Humidity           10
Cloudiness         10
Wind Speed         10
Country            10
Date               10
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Using '.copy()' will help not interupt my 'ideal_weather' dataframe:
hotel_df = ideal_weather.copy()
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
106,Awjilah,29.11,21.29,72.18,62,0,8.61,LY,1594435388
113,Mednogorsk,51.41,57.59,79.95,56,0,1.97,RU,1594435515
141,Marsá Maţrūḩ,31.35,27.25,71.60,83,0,5.82,EG,1594435522
214,Jalu,29.03,21.55,72.28,62,0,8.10,LY,1594435541
225,Komsomolets,53.75,62.07,79.97,65,0,4.27,KZ,1594435543
302,Corinth,37.94,22.96,74.52,44,0,1.01,GR,1594435561
360,Al Bardīyah,31.76,25.09,71.01,81,0,8.59,LY,1594435463
371,Lazurne,46.08,32.53,72.57,66,0,7.45,UA,1594435576
378,Derzhavīnsk,51.10,66.32,76.37,57,0,7.54,KZ,1594435578
444,Ben Gardane,33.14,11.22,72.41,82,0,5.01,TN,1594435592


In [11]:
# Adding a new column for 'Hotel Name':
hotel_df['Hotel Name'] = ""
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
106,Awjilah,29.11,21.29,72.18,62,0,8.61,LY,1594435388,
113,Mednogorsk,51.41,57.59,79.95,56,0,1.97,RU,1594435515,
141,Marsá Maţrūḩ,31.35,27.25,71.60,83,0,5.82,EG,1594435522,
214,Jalu,29.03,21.55,72.28,62,0,8.10,LY,1594435541,
225,Komsomolets,53.75,62.07,79.97,65,0,4.27,KZ,1594435543,
302,Corinth,37.94,22.96,74.52,44,0,1.01,GR,1594435561,
360,Al Bardīyah,31.76,25.09,71.01,81,0,8.59,LY,1594435463,
371,Lazurne,46.08,32.53,72.57,66,0,7.45,UA,1594435576,
378,Derzhavīnsk,51.10,66.32,76.37,57,0,7.54,KZ,1594435578,
444,Ben Gardane,33.14,11.22,72.41,82,0,5.01,TN,1594435592,


In [12]:
# Searching for the closest hotels to the cities that meet the weather criteria:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "key": g_key,
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel"
}

for index, row in hotel_df.iterrows():
    
    ideal_lat = row['Latitude']
    ideal_long = row['Longitude']
    params['location'] = f"{ideal_lat}, {ideal_long}"
    response = requests.get(url, params=params).json()
    results = response['results']
    
# pprint(results)    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "None available"
        
        pass      

In [13]:
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
106,Awjilah,29.11,21.29,72.18,62,0,8.61,LY,1594435388,None available
113,Mednogorsk,51.41,57.59,79.95,56,0,1.97,RU,1594435515,"Hotel ""Central"""
141,Marsá Maţrūḩ,31.35,27.25,71.60,83,0,5.82,EG,1594435522,Beau Site
214,Jalu,29.03,21.55,72.28,62,0,8.10,LY,1594435541,Jalu Hotel
225,Komsomolets,53.75,62.07,79.97,65,0,4.27,KZ,1594435543,Gostinitsa Zarya
302,Corinth,37.94,22.96,74.52,44,0,1.01,GR,1594435561,Hotel King Saron Ξενοδοχείο
360,Al Bardīyah,31.76,25.09,71.01,81,0,8.59,LY,1594435463,Hotel Burdy
371,Lazurne,46.08,32.53,72.57,66,0,7.45,UA,1594435576,Пансионат Panamera
378,Derzhavīnsk,51.10,66.32,76.37,57,0,7.54,KZ,1594435578,Gostinitsa Galaktika
444,Ben Gardane,33.14,11.22,72.41,82,0,5.01,TN,1594435592,Hôtel Palace


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Add marker layer on top of heat map
# Display figure
fig = gmaps.figure(zoom_level=2, center=(20,30))
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))